In [2]:
from numpy import loadtxt
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from scipy.stats import zscore
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from keras.callbacks import ModelCheckpoint
import tensorflow as tf
from sklearn.model_selection import KFold
from tensorflow.keras.layers import Dense, Activation
from keras.utils import to_categorical
from sklearn.model_selection import StratifiedKFold

Using TensorFlow backend.


In [3]:
# LER OS DADOS E SEPARA-LOS
dataset = pd.read_csv('dadosCORRETOS.csv', header=1)

In [4]:
x = dataset.iloc[:,3:54676].values
y = dataset.iloc[:,2].values
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

In [ ]:
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
# Cross-Validate
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=7)
oos_y = []
oos_pred = []
models = []
cvscores = []
history = []
fold = 0
for train, test in kf.split(X_train, y_train):
    fold+=1
    print(f"Fold #{fold}")
    filepath= 'teste2/'+str(fold)+"-Correto4.hdf5"
    filepath2= 'teste2/'+str(fold)+"-Correto5.hdf5"
    filepath3= 'teste2/'+str(fold)+"-Correto6.hdf5"
    filepath4= 'teste2/'+str(fold)+"-Correto7.hdf5"
    mcp_save = ModelCheckpoint(filepath ,save_best_only=True, verbose=0, monitor='val_loss', mode='min')
    acura_save = ModelCheckpoint(filepath2 ,save_best_only=True, verbose=0, monitor='val_accuracy', mode='max')
    mcp_savew = ModelCheckpoint(filepath3 ,save_best_only=True, verbose=0, monitor='loss', mode='min')
    acura_savew = ModelCheckpoint(filepath4 ,save_best_only=True, verbose=0, monitor='accuracy', mode='max')
    
    model = tf.keras.Sequential()
    model.add(Dense(64, input_dim=x.shape[1], activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='Adam', metrics=['accuracy'])
    #   
    history.append(model.fit(X_train[train], y_train[train], validation_data=(X_train[test], y_train[test]), 
        # salvar parametros melhor acuracy # callbacks= [acura_save],
                             verbose=2, batch_size=200, epochs=100))
    
    pred = model.predict(X_test)
    
    oos_y.append(y_test)
    oos_pred.append(pred)
    models.append(model)
    score = np.sqrt(metrics.mean_squared_error(pred,y_test))
    print(f"Fold score (RMSE): {score}")
    
oos_y = np.concatenate(oos_y)
oos_pred = np.concatenate(oos_pred)
score = np.sqrt(metrics.mean_squared_error(oos_pred,oos_y))
print(f"Final, out of sample score (RMSE): {score}")

Using TensorFlow backend.


In [ ]:
# Mostrar melhor resultado obtidos em cada fold

for result in history:
    validation_acc = np.amax(result.history['val_accuracy'])
    print(validation_acc)

In [ ]:
# Mostrar acuracia de cada fold nos dados de teste

for model in models:
    _, accuracy = model.evaluate(X_test, y_test, verbose = 0)
    print('Accuracy: %.2f' % (accuracy*100))

In [ ]:
def plot_history(history):
    loss_list = [s for s in history.history.keys() if 'loss' in s and 'val' not in s]
    val_loss_list = [s for s in history.history.keys() if 'loss' in s and 'val' in s]
    acc_list = [s for s in history.history.keys() if 'acc' in s and 'val' not in s]
    val_acc_list = [s for s in history.history.keys() if 'acc' in s and 'val' in s]
    
    if len(loss_list) == 0:
        print('Loss is missing in history')
        return 
    
    ## As loss always exists
    epochs = range(1,len(history.history[loss_list[0]]) + 1)
    
    ## Loss
    plt.figure(1)
    for l in loss_list:
        plt.plot(epochs, history.history[l], 'b', label='Training loss (' + str(str(format(history.history[l][-1],'.5f'))+')'))
    for l in val_loss_list:
        plt.plot(epochs, history.history[l], 'g', label='Validation loss (' + str(str(format(history.history[l][-1],'.5f'))+')'))
    
    plt.title('Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    
    ## Accuracy
    plt.figure(2)
    for l in acc_list:
        plt.plot(epochs, history.history[l], 'b', label='Training accuracy (' + str(format(history.history[l][-1],'.5f'))+')')
    for l in val_acc_list:    
        plt.plot(epochs, history.history[l], 'g', label='Validation accuracy (' + str(format(history.history[l][-1],'.5f'))+')')

    plt.title('Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.show()

In [ ]:
# Plotar grafico de cada fold

for historico in history:
    plot_history(historico)